In [1]:
import requests
import json
import pandas as pd
import os
import time
import re
import ast

In [2]:
games = pd.read_csv('../data/rawg_games_raw_data.csv')
developers = pd.read_csv('../data/rawg_developer_raw_data.csv')

## Cleaning Games DF

In [4]:
games

,Unnamed: 0,id,slug,name,released,tba,background_image,rating,rating_top,ratings,...,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating
0,0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290376, 'store': {'id': 1, 'name': 'St...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
1,1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.64,5,"[{'id': 5, 'title': 'exceptional', 'count': 53...",...,0f0f0f,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
2,2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.59,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
3,3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.57,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
4,4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 24...",...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 33824, 'store': {'id': 7, 'name': 'Xbo...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,21970,crimson-earth,Crimson Earth,2017-05-09,False,https://media.rawg.io/media/screenshots/507/50...,0.00,0,"[{'id': 1, 'title': 'skip', 'count': 2, 'perce...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 24731, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0
19996,19996,21553,slamit-pinball-big-score,SlamIt Pinball Big Score,2009-02-23,False,https://media.rawg.io/media/screenshots/ad7/ad...,0.00,0,"[{'id': 4, 'title': 'recommended', 'count': 1,...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 11, 'name': 'Arcade', 'slug': 'arcade'...","[{'id': 24251, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'n

In [5]:
def remove_string(list_str):
        extracted_list = ast.literal_eval(list_str)
        return extracted_list

In [6]:
games.columns

Index(['Unnamed: 0', 'id', 'slug', 'name', 'released', 'tba',
       'background_image', 'rating', 'rating_top', 'ratings', 'ratings_count',
       'reviews_text_count', 'added', 'added_by_status', 'metacritic',
       'playtime', 'suggestions_count', 'updated', 'user_game',
       'reviews_count', 'saturated_color', 'dominant_color', 'platforms',
       'parent_platforms', 'genres', 'stores', 'clip', 'tags', 'esrb_rating',
       'short_screenshots', 'community_rating'],
      dtype='object')

In [7]:
games = games.drop(columns = ['background_image', 'saturated_color', 'dominant_color', 'short_screenshots', 'clip'])
games['ratings'] = games['ratings'].apply(remove_string)
games.head(2)

,Unnamed: 0,id,slug,name,released,tba,rating,rating_top,ratings,ratings_count,...,updated,user_game,reviews_count,platforms,parent_platforms,genres,stores,tags,esrb_rating,community_rating
0,0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",7118,...,2025-06-10T15:19:04,NaN,7232,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290376, 'store': {'id': 1, 'name': 'St...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN
1,1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.64,5,"[{'id': 5, 'title': 'exceptional', 'count': 53...",6913,...,2025-06-10T15:17:24,NaN,7026,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN


In [8]:
games

,Unnamed: 0,id,slug,name,released,tba,rating,rating_top,ratings,ratings_count,...,updated,user_game,reviews_count,platforms,parent_platforms,genres,stores,tags,esrb_rating,community_rating
0,0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",7118,...,2025-06-10T15:19:04,NaN,7232,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290376, 'store': {'id': 1, 'name': 'St...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN
1,1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.64,5,"[{'id': 5, 'title': 'exceptional', 'count': 53...",6913,...,2025-06-10T15:17:24,NaN,7026,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN
2,2,4200,portal-2,Portal 2,2011-04-18,False,4.59,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",5908,...,2025-06-09T01:03:22,NaN,5963,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...",NaN
3,3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,3.57,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",3574,...,2025-06-09T09:26:41,NaN,3612,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...","[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN
4,4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 24...",4007,...,2025-06-09T22:30:53,NaN,4039,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 33824, 'store': {'id': 7, 'name': 'Xbo...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,21970,crimson-earth,Crimson Earth,2017-05-09,False,0.00,0,"[{'id': 1, 'title': 'skip', 'count': 2, 'perce...",4,...,2019-09-17T13:41:43,NaN,4,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 24731, 'store': {'id': 1, 'name': 'Ste...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,0.0
19996,19996,21553,slamit-pinball-big-score,SlamIt Pinball Big Score,2009-02-23,False,0.00,0,"[{'id': 4, 'title': 'recommended', 'count': 1,...",1,...,2019-08-28T22:07:58,NaN,1,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 11, 'name': 'Arcade', 'slug': 'arcade'...","[{'id': 24251, 'store': {'id': 1, 'name': 'Ste...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,0.0
19997,19997,21390,ford-racing-off-road,Ford Racing Off Road,2008-08-26,False,0.00,0,"[{'id': 3, 'title': 'meh', 'count': 3, 'percen...",5,...,2019-08-28T22:08:04,NaN,4,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 14, 'name': 'Simulation', 'slug': 'sim...","[{'id': 24066, 'store': {'id': 1, 'name': 'Ste...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 1, 'name': 'Everyone', 'slug': 'everyone'

In [9]:
games = games[~games['ratings'].apply(lambda x: x == [])]

In [10]:
games = games.reset_index(drop=True)

In [11]:
pd.json_normalize(games['ratings'])[0][0]

{'id': 5, 'title': 'exceptional', 'count': 4267, 'percent': 59.0}

In [12]:
pd.json_normalize(games['ratings'][2]).set_index('title', drop=True)[['count']].T.reset_index()

title,index,exceptional,recommended,meh,skip
0,count,4134,1500,180,149


<h3>Cleaning Ratings Column</h3>

In [14]:
concat_list = []

for i in range(len(games['ratings'])):
    concat_list.append(
        pd.json_normalize(
            games['ratings'][i]
        ).set_index('title', drop=True)[['count']].T
    )
    print(f'finished at index {i}')
ratings_df = pd.concat(concat_list)

renamed = {'exceptional':'rating_exceptional','recommended':'rating_recommended','meh':'rating_meh','skip':'rating_skip'}
ratings_df = ratings_df.rename(columns=renamed).reset_index(drop=True)

games = pd.concat([games, ratings_df], axis=1).drop(columns=['ratings', 'esrb_rating'])

finished at index 0
finished at index 1
finished at index 2
finished at index 3
finished at index 4
finished at index 5
finished at index 6
finished at index 7
finished at index 8
finished at index 9
finished at index 10
finished at index 11
finished at index 12
finished at index 13
finished at index 14
finished at index 15
finished at index 16
finished at index 17
finished at index 18
finished at index 19
finished at index 20
finished at index 21
finished at index 22
finished at index 23
finished at index 24
finished at index 25
finished at index 26
finished at index 27
finished at index 28
finished at index 29
finished at index 30
finished at index 31
finished at index 32
finished at index 33
finished at index 34
finished at index 35
finished at index 36
finished at index 37
finished at index 38
finished at index 39
finished at index 40
finished at index 41
finished at index 42
finished at index 43
finished at index 44
finished at index 45
finished at index 46
finished at index 47
fi

<h3>Cleaning ESRB Rating Column</h3>

In [16]:
games['esrb_rating'] = games['esrb_rating'].apply(lambda x: remove_string(x) if pd.notna(x) else x)

In [17]:
games['esrb_rating'] = pd.json_normalize(games['esrb_rating'])[['name']].rename(columns={'name':'esrb_rating'})

<h3>Cleaning Genres Column</h3>

In [19]:
games['genres'] = games['genres'].apply(remove_string)

In [20]:
outer_genre_names = []

for j in range(len(games['genres'])):
    inner_genre_names = []
    
    for i in games['genres'].iloc[j]:
        inner_genre_names.append(i['name'])
    
    total_genre_name = '_'.join(inner_genre_names)

    outer_genre_names.append(total_genre_name)

games['genres'] = pd.DataFrame(outer_genre_names, columns = ['genre'])

<h3>Cleaning Platform Column</h3>

In [25]:
games['parent_platforms'] = games['parent_platforms'].apply(remove_string)

In [37]:
games['parent_platforms']

0        [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
1        [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
2        [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
3        [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
4        [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
                               ...                        
18957    [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
18958    [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
18959    [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
18960    [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
18961    [{'platform': {'id': 1, 'name': 'PC', 'slug': ...
Name: parent_platforms, Length: 18962, dtype: object

In [29]:
pd.json_normalize(games['parent_platforms'][0])

,platform.id,platform.name,platform.slug
0,1,PC,pc
1,2,PlayStation,playstation
2,3,Xbox,xbox


In [22]:
pd.get_dummies(pd.json_normalize(
    remove_string(games['parent_platforms'][0]))[['platform.name']], columns = ['platform.name']
)

,platform.name_PC,platform.name_PlayStation,platform.name_Xbox
0,True,False,False
1,False,True,False
2,False,False,True


In [ ]:
games = pd.concat([games, ratings_df, esrb_rating], axis=1).drop(columns=['ratings', 'esrb_rating'])

## Cleaning Developers

In [ ]:
developers

In [ ]:
developers['games'] = developers['games'].apply(remove_string)
developers

In [ ]:
l = []
for i in range(len(developers)):
    df = pd.json_normalize(developers['games'][i])
    df['company'] = developers['name'][i]
    l.append(df)

developer_df = pd.concat(l).reset_index()
developer_df